# REPORT CHECK

Used to double-check the results for the standard data case

In [64]:
import pandas as pd
import functools as ft
import itertools as it
import json

In [65]:
# PATH TO JSON 
RAW_EVENTS_INPUT_FILE = "../data/list_data.json"
REPORT_RESULT_JSON_INPUT = "../data/response.json"

# EXCEL OUTPUT
CALC_REPORT_EXCEL_OUTPUT =  "../data/report_demo.xlsx"

## Data Report Generation

Script to read all events information at once to check if the report results are to be trusted

### Input

In [66]:
df = pd.read_json(RAW_EVENTS_INPUT_FILE, orient="records")
df

,date,quantity,item,price
0,2023-01-01 00:32:24+00:00,8,cherry,2.3
1,2023-01-01 01:11:10+00:00,8,mango,6.0
2,2023-01-01 01:15:09+00:00,7,watermelon,4.7
3,2023-01-01 01:37:20+00:00,10,apple,2.0
4,2023-01-01 02:46:13+00:00,4,apple,2.0
...,...,...,...,...
9930,2023-12-30 20:59:40+00:00,10,strawberry,5.2
9931,2023-12-30 21:13:13+00:00,5,strawberry,5.2
9932,2023-12-30 21:58:51+00:00,6,banana,1.5
9933,2023-12-30 22:12:52+00:00,7,cherry,2.3


### Calculations

In [67]:
def per_item_agg(df):
    return (
        df
        .assign(
            revenue=lambda df_: df_["quantity"] * df_["price"]
        )
        .groupby("item")
        .agg(
            total_revenue=("revenue", "sum"),
            total_sales=("date", "count"),
            total_quantity=("quantity", "sum"),
        )
        .assign(
            average_per_sale=lambda df_: df_["total_quantity"] / df_["total_sales"],
        )
        .drop(columns=["total_sales"])
    )

per_item_agg = df.pipe(per_item_agg)
per_item_agg

,total_revenue,total_quantity,average_per_sale
item,,,
apple,11162.0,5581,5.542205
avocado,27658.8,5319,5.433095
banana,7855.5,5237,5.483770
cherry,12907.6,5612,5.475122
lemon,17846.4,5577,5.494581
mango,32724.0,5454,5.432271
papaya,39587.9,5423,5.455734
pineapple,36153.2,5396,5.632568
strawberry,28698.8,5519,5.453557


In [68]:
def per_item_month_agg(df):
    return (
        df
        .assign(
            revenue=lambda df_: df_["quantity"] * df_["price"],
            year=lambda df_: df_["date"].dt.year,
            month=lambda df_: df_["date"].dt.month,
        )
        .groupby(["item", "year", "month"])
        .agg(
            total_revenue=("revenue", "sum"),
            total_sales=("date", "count"),
            total_quantity=("quantity", "sum"),
        )
        .assign(
            average_per_sale=lambda df_: df_["total_quantity"] / df_["total_sales"],
        )
        .drop(columns=["total_sales"])
    )

per_item_month_agg = df.pipe(per_item_month_agg)
per_item_month_agg

total_revenue  total_quantity  average_per_sale
item       year month                                                 
apple      2023 1              836.0             418          5.428571
                2              890.0             445          5.855263
                3              856.0             428          5.706667
                4              760.0             380          4.935065
                5             1036.0             518          5.569892
...                              ...             ...               ...
watermelon 2023 8             1748.4             372          5.239437
                9             2209.0             470          5.595238
                10            2204.3             469          5.097826
                11            2232.5             475          5.722892
                12            2063.3             439          6.097222

[120 rows x 3 columns]

### Output

In [69]:
with pd.ExcelWriter(CALC_REPORT_EXCEL_OUTPUT) as writer:
    per_item_agg.to_excel(writer, sheet_name="per_item")
    per_item_month_agg.to_excel(writer, sheet_name="per_item_month")

## Resulsts assert

In [70]:
@ft.cache
def load_report():
    with open(REPORT_RESULT_JSON_INPUT) as f:
        return json.load(f)

load_report()

{'strawberry': {'total_quantity': 5519,
  'average_per_sale': 5,
  'total_revenue': 28698,
  'monthly': {'2023-01': {'total_quantity': 492,
    'average_per_sale': 5,
    'total_revenue': 2558},
   '2023-02': {'total_quantity': 458,
    'average_per_sale': 5,
    'total_revenue': 2381},
   '2023-03': {'total_quantity': 598,
    'average_per_sale': 5,
    'total_revenue': 3109},
   '2023-04': {'total_quantity': 446,
    'average_per_sale': 5,
    'total_revenue': 2319},
   '2023-05': {'total_quantity': 506,
    'average_per_sale': 5,
    'total_revenue': 2631},
   '2023-06': {'total_quantity': 472,
    'average_per_sale': 5,
    'total_revenue': 2454},
   '2023-07': {'total_quantity': 414,
    'average_per_sale': 5,
    'total_revenue': 2152},
   '2023-08': {'total_quantity': 396,
    'average_per_sale': 4,
    'total_revenue': 2059},
   '2023-09': {'total_quantity': 349,
    'average_per_sale': 5,
    'total_revenue': 1814},
   '2023-10': {'total_quantity': 520,
    'average_per_sale':

In [71]:
def agg_report(d):
    keys = ["total_quantity", "average_per_sale", "total_revenue"]
    def sel_agg_json(j):
        return { k: j[k] for k in keys }

    j_reports = d.values()
    j_agg_reports = map(sel_agg_json, j_reports)
    return dict(zip(d.keys(), j_agg_reports))

def agg_report_df(agg_report):
    return (
        pd.DataFrame.from_dict(agg_report, orient="index")
        .astype({
            "total_quantity": "int",
            "average_per_sale":	"int",
            "total_revenue": "int",
        })
        .sort_index()
    )

result_agg = agg_report_df(agg_report(load_report()))
result_agg

,total_quantity,average_per_sale,total_revenue
apple,5581,5,11162
avocado,5319,5,27658
banana,5237,5,7855
cherry,5612,5,12907
lemon,5577,5,17846
mango,5454,5,32724
papaya,5423,5,39587
pineapple,5396,5,36153
strawberry,5519,5,28698
watermelon,5559,5,26127


In [72]:
def compare_agg_report(res_df, calc_df):

    _calc_df = (
        calc_df
        .astype("int")
        .reindex(columns=res_df.columns)
    )
    return res_df.compare(_calc_df)


compare_agg_report(result_agg, per_item_agg)

Empty DataFrame
Columns: []
Index: []

In [73]:
load_report()

{'strawberry': {'total_quantity': 5519,
  'average_per_sale': 5,
  'total_revenue': 28698,
  'monthly': {'2023-01': {'total_quantity': 492,
    'average_per_sale': 5,
    'total_revenue': 2558},
   '2023-02': {'total_quantity': 458,
    'average_per_sale': 5,
    'total_revenue': 2381},
   '2023-03': {'total_quantity': 598,
    'average_per_sale': 5,
    'total_revenue': 3109},
   '2023-04': {'total_quantity': 446,
    'average_per_sale': 5,
    'total_revenue': 2319},
   '2023-05': {'total_quantity': 506,
    'average_per_sale': 5,
    'total_revenue': 2631},
   '2023-06': {'total_quantity': 472,
    'average_per_sale': 5,
    'total_revenue': 2454},
   '2023-07': {'total_quantity': 414,
    'average_per_sale': 5,
    'total_revenue': 2152},
   '2023-08': {'total_quantity': 396,
    'average_per_sale': 4,
    'total_revenue': 2059},
   '2023-09': {'total_quantity': 349,
    'average_per_sale': 5,
    'total_revenue': 1814},
   '2023-10': {'total_quantity': 520,
    'average_per_sale':

In [74]:
def month_agg_report(d):
    def flat_json(report, item):
        monthly = report["monthly"]
        for m_str, report in monthly.items():
            month_dt = pd.to_datetime(f"{m_str}-01")
            yield {
                "month_dt": month_dt,
                "item": item,
                **report
            }
    
    records_its = map(flat_json, d.values(), d.keys())
    return it.chain.from_iterable(records_its)

def month_agg_report_df(agg_report):
    return (
        pd.DataFrame.from_records(agg_report)
        .astype({
            "month_dt": "datetime64[ns]",
            "item": "str",
            "total_quantity": "int",
            "average_per_sale":	"int",
            "total_revenue": "int",
        })
        .assign(
            year=lambda df_: df_["month_dt"].dt.year,
            month=lambda df_: df_["month_dt"].dt.month,
        )
        .drop(columns=["month_dt"])
        .set_index(["item", "year", "month"])
        .sort_index()
    )

month_result_agg = month_agg_report_df(month_agg_report(load_report()))
month_result_agg.dtypes

total_quantity      int64
average_per_sale    int64
total_revenue       int64
dtype: object

In [75]:
per_item_month_agg.dtypes

total_revenue       float64
total_quantity        int64
average_per_sale    float64
dtype: object

In [76]:
def compare_months_report(result, calc):
    _calc = (
        calc
        .astype("int")
        .reindex(columns=result.columns)
    )

    return (
        result
        .compare(_calc, result_names=("result", "calc"))
        .loc[lambda df_: ~df_.isna().all(axis=1)]
    )

compare_months_report(month_result_agg, per_item_month_agg)

item,year,month
